# Helmholtz Iterative Refinement: Error-Centric Dataset Generation
This notebook generates a paired dataset for training a Neural Operator $T$. 
Instead of mapping sources to solutions, we map **High-Frequency Residuals** to **Low-Frequency Corrections**.

### The Logic (from "Assume Love" notes):
1. **Residual:** $r^{(k)} = f - L_{\omega} u^{(k)}$
2. **Transfer:** $\tilde{e}^{(k)} = \text{NeuralOperator}(r^{(k)}, \omega, \omega')$
3. **Update:** $u^{(k+1)} = u^{(k)} + \text{Upsample}(\tilde{e}^{(k)})$

In [1]:
print("Hello, World!")

Hello, World!


In [ ]:
import numpy as np
import json
import shutil
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt

# Custom project imports
from core.grid import Grid2D
from algorithm.dataset import solve_one_sample, append_jsonl

# Global Settings
OUT_ROOT = Path("data/residual_correction_dataset")
if OUT_ROOT.exists():
    shutil.rmtree(OUT_ROOT)
OUT_ROOT.mkdir(parents=True)

# Physical Constants
K_RATIO = 2.0  # Fixed ratio between high and low frequency
N_PHYS = 500   # Fixed 500x500 grid
LX, LY = 1.0, 1.0

ModuleNotFoundError: No module named 'core'